In [1]:
# Import packages
import operator
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import polyglot
from polyglot.downloader import downloader
from polyglot.text import Text
import wikipedia
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from googletrans import Translator
translator = Translator()
from polyglot.detect import Detector
from datetime import datetime, date
import requests
from bs4 import BeautifulSoup

In [2]:
# Path of the cross lingual word embeddings (CLWEs)
en_path = 'wiki.multi.en.vec.txt'
sp_path = 'wiki.multi.es.vec.txt'
it_path = 'wiki.multi.it.vec.txt'
est_path = 'wiki.multi.et.vec.txt'
rus_path = 'wiki.multi.ru.vec.txt'

In [3]:
# A function that transforms the parallel data into vectors
def make_vectors(directory):
    table = dict()
    with open(directory, 'r', encoding='utf-8', errors='ignore') as f:
        next(f)
        vectors = []
        for i, line in enumerate(f):
            word,vect = line.rstrip().split(" ",1)
            vect = np.fromstring(vect, sep=' ')
            table[word] = vect
    return table

In [4]:
# Obtaining the CLWEs for english, spanish, italian, estonian, russian
table_eng = make_vectors(en_path)
table_sp = make_vectors(sp_path)
table_it = make_vectors(it_path)
table_est = make_vectors(est_path)
table_rus = make_vectors(rus_path)

In [13]:
# function that transforms tokens into vectors
def token2multi_vectors(token):
        # Assign the multilingual vectors to the input words
        # assign a zero confidence to stopwords
        try:
            vect = table_eng[token].reshape(1,300)   
        except:
            vect = UNK
        try:
            vect_sp = table_sp[token].reshape(1,300)
        except:
            vect_sp =UNK
        try:
            vect_it = table_it[token].reshape(1,300)
        except:
            vect_it = UNK
        try:
            vect_rus = table_rus[token].reshape(1,300)
        except:
            vect_rus = UNK
        try:
            vect_est = table_est[token].reshape(1,300)
        except:
            vect_est = UNK
        return vect, vect_sp, vect_it, vect_rus, vect_est

In [14]:
# function that intercepts the vectors related to the time of the day
def when_def(when_dict, conf_value, languages):
    sim_today = []
    for key, value in when_dict.items():
        value = value.reshape(1,300)
        for lang in languages:
            sim = cosine_similarity(lang, value)
            if sim > conf_value:
                sim_today.append(sim)
    return sum(sim_today)

In [15]:
def prep_city(from_phrase, parola, lang):
    prep = 0
    print(from_phrase) 
    city = input('YOU: ')
    tok = city.lower().split()
    for i in range(len(tok)):
        vect, vect_sp, vect_it,vect_rus,vect_est = token2multi_vectors(tok[i])
        if lang == 'en':
            if cosine_similarity(vect, table_eng[parola].reshape(1,300)) > 0.8:
                prep += 1
        elif lang == 'es':
            if cosine_similarity(vect_sp, table_eng[parola].reshape(1,300)) > 0.3:
                prep += 1
        elif lang == 'it':
            if cosine_similarity(vect_it, table_eng[parola].reshape(1,300)) > 0.3:
                prep += 1
        elif lang == 'et':
            if cosine_similarity(vect_est, table_eng[parola].reshape(1,300)) > 0.3:
                prep += 1
        elif lang == 'ru':
            if cosine_similarity(vect_rus, table_eng[parola].reshape(1,300)) > 0.3:
                prep += 1
    if prep > 0: 
        try:
            if tok[2] != None:
                CITY = tok[1].capitalize() + " " + tok[2].capitalize()      
        except:
            CITY = tok[1].capitalize()    
    else:
        CITY = city.capitalize()
    return CITY

In [16]:
# WeatherBot answer

def answer_weather(today, tomorrow, phrase_locality, phrase_when,  TODAY, TOMORROW, CITY, sorry_answer, lang):
    BOT = 'BOT:'
    if CITTA == False:
        CITY = prep_city(phrase_locality, 'in', lang)          
    else:
        try:
            CITY = CITY.split()
            if CITY[1] != None:
                CITY = CITY[0].capitalize() + " " + CITY[1].capitalize()
        except:
            CITY = "".join(CITY).capitalize()
    
    if today == False and tomorrow == False:
        ty = 0
        tw = 0
        print(BOT, phrase_when)
        when = input('YOU: ').lower().split()
        for i in range(len(when)):
            vect, vect_sp, vect_it,vect_rus,vect_est = token2multi_vectors(when[i])
            languages = [vect, vect_est, vect_it, vect_rus, vect_sp]
            oggi = when_def(today_key, 0.5, languages)
            domani = when_def(tomorrow_key, 0.5, languages)
            if oggi > 0 and oggi > domani:
                ty +=1
            if domani > 0 and domani > oggi:
                tw +=1

        if ty > 0:
            today = TODAY
            try:
                url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
                res = requests.get(url)
                data = res.json()
                temp = data['main']['temp']
                sky = data['weather'][0]['description']
                if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  CITY, today), dest=lang).text)
                else:
                    print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,CITY, today))
            except:
                print(BOT,sorry_answer)

        elif tw > 0:
            tomorrow = TOMORROW
            try:
                url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
                res = requests.get(url)
                data = res.json()
                temp = data['main']['temp']
                sky = data['weather'][0]['description']
                if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  city, tomorrow), dest=lang).text)
                else:
                    print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,  CITY, tomorrow))
            except:
                print(BOT, sorry_answer)

        else:
            try:
                url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
                res = requests.get(url)
                data = res.json()
                temp = data['main']['temp']
                sky = data['weather'][0]['description']
                if lang != 'en':
                    print(BOT, translator.translate("In this day there is {} and there are {} degrees in {}".format(sky, temp, 
                    CITY), dest=lang).text)
                else:
                    print(BOT, "In this day there is {} and there are {} degrees in {}".format(sky, temp,  CITY))
            except:
                print(BOT, sorry_answer)

    elif today == True and tomorrow == False:
        today = TODAY
        try:
            url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
            res = requests.get(url)
            data = res.json()
            temp = data['main']['temp']
            sky = data['weather'][0]['description']
            if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  CITY, today), dest=lang).text)
            else:
                print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,  CITY, today))
        except:
            print(BOT, sorry_answer)

    elif tomorrow == True:
        tomorrow = TOMORROW
        
        try:
            url = "https://openweathermap.org/data/2.5/weather?q={}&appid=b6907d289e10d714a6e88b30761fae22".format(CITY)
            res = requests.get(url)
            data = res.json()
            temp = data['main']['temp']
            sky = data['weather'][0]['description']
            if lang != 'en':
                    print(BOT, translator.translate("There is {} and there are {} degrees in {} {}".format(sky,
                    temp,  CITY, tomorrow), dest=lang).text)
            else:
                print(BOT, "There is {} and there are {} degrees in {} {}".format(sky, temp,  CITY, tomorrow))
        except:
            print(BOT, sorry_answer)

In [17]:
# Wiki answers

def answer_wiki(frase_sorry, frase_noinfo):
    
    if WHAT == True:
        try:
            res = ".".join(wikipedia.summary(wikipedia.search(topic, results=1)).split('.')[0:2])
            print('BOT:', res)
        except:
            print(frase_sorry)
            response = input('YOU: ').lower()
            try:
                res = ".".join(wikipedia.summary(wikipedia.search(response, results=1)).split('.')[0:2])
                print("BOT:", res)
            except:
                print(frase_noinfo, response)
    else:   
        try:
            if person == True:
                res = ".".join(wikipedia.summary(wikipedia.search(PERSON, results=1)).split('.')[0:2])
                print('BOT:', res)
            elif CITTA == True:
                res = ".".join(wikipedia.summary(wikipedia.search(CITY, results=1)).split('.')[0:2])
                print('BOT:', res)
            else:
                print(frase_sorry)
                response = input('YOU: ').lower()
                try:
                    res = ".".join(wikipedia.summary(wikipedia.search(response, results=1)).split('.')[0:2])
                    print("BOT:", res)
                except:
                    print(frase_noinfo, response)
        except:
            print(frase_sorry)
            response = input('YOU: ').lower()
            try:
                res = ".".join(wikipedia.summary(wikipedia.search(response, results=1)).split('.')[0:2])
                print("BOT:", res)
            except:
                print(frase_noinfo, response)


In [18]:
# Travelbot answers
def answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, lang, destination_phrase, from_city_phrase, sorry_phrase, when_phrase, oggi, domani):

    if CITY != None:
        CITY = CITY.split()
        try:
            if CITY[1] != None:
                CITY = CITY[0].capitalize() + " " + CITY[1].capitalize()
        except:
            CITY = "".join(CITY).capitalize()
    if CITY2 != None:
        CITY2 = CITY2.split()
        try:
            if CITY2[1] != None:
                CITY2 = CITY2[0].capitalize() + " " + CITY2[1].capitalize()
        except:
            CITY2 = "".join(CITY2).capitalize()

    if CITY2 !=None:
        if leaving == arriving:
            print(destination_phrase)
            tok = input('YOU: ').lower().capitalize().split()
            try:
                if tok[1] != None:
                    tok = tok[0].capitalize() + " " + tok[1].capitalize()  
            except:
                tok = tok[0].capitalize()  
            if tok[:3] == CITY[:3]:
                arriving = CITY
                leaving = CITY2
            else:
                arriving = CITY2
                leaving = CITY
        
        if leaving != None:
            leaving = leaving
        else:
            where2start = prep_city(from_city_phrase, 'from', lang)
            # we just confront the first 3 letters of the 2 cities (in case of languages wt different suffixies)
            if where2start[:3] == CITY2[:3]:
                arriving = CITY
                leaving = CITY2
            elif where2start[:3] == CITY[:3]:
                arriving = CITY2
                leaving = CITY
            else:
                print(sorry_phrase)
                today = True

        if leaving != None and arriving == None:
            if leaving == CITY:
                arriving = CITY2
            elif leaving ==CITY2:
                arriving = CITY
            else:
                print(destination_phrase)
                arriving = input('YOU: ').lower().capitalize().split()
                try:
                    if arriving[1] != None:
                        arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
                except:
                    arriving = arriving[0].capitalize() 

        if arriving != None and leaving == None:
            if arriving == CITY:
                leaving = CITY2
            elif arriving == CITY:
                leaving = CITY
            else:
                leaving = prep_city(from_city_phrase, 'from', lang)

    if CITY != None and CITY2 == None:
        if arriving != None:
            arriving = CITY
            leaving = prep_city(from_city_phrase, 'from', lang)
        elif leaving != None:
            leaving = CITY
            print(destination_phrase)
            arriving = input('YOU: ').lower().capitalize().split()
            try:
                if arriving[1] != None:
                    arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
            except:
                arriving = arriving[0].capitalize()
        else:
            where2start = prep_city(from_city_phrase, 'from', lang)
            # we just confront the first 3 letters of the 2 cities (in case of languages wt different suffixies)
            if where2start[:3] == CITY[:3]:
                leaving = CITY
                arriving = input('YOU: ').lower().capitalize().split()
                try:
                    if arriving[1] != None:
                        arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
                except:
                    arriving = arriving[0].capitalize()
                    
            else:
                arriving = CITY
                leaving = where2start
            

    elif CITY == None:
        leaving = prep_city(from_city_phrase, 'from', lang)
        
        arriving = input('YOU: ').lower().capitalize().split()
        try:
            if arriving[1] != None:
                arriving = arriving[0].capitalize() + " " + arriving[1].capitalize()  
        except:
            arriving = arriving[0].capitalize()

    if today == False and tomorrow == False:
        ty = 0
        tw = 0
        print(when_phrase)
        day = input('YOU: ').lower()
        for i in range(len(day)):
            vect, vect_sp, vect_it,vect_rus,vect_est = token2multi_vectors(day[i])
            languages = [vect, vect_est, vect_it, vect_rus, vect_sp]
            oggi = when_def(today_key, 0.5, languages)
            domani = when_def(tomorrow_key, 0.5, languages)
            if oggi > 0 and oggi > domani:
                ty +=1
            if domani > 0 and domani > oggi:
                tw +=1
        if ty > 0:
            today = oggi
        elif tw > 0:
            tomorrow = domani
    else:
        if today == True:
            day = oggi
        elif tomorrow == True:
            day = domani
    
    if lang != 'en':
        if lang == 'ru':
            print("Я нахожу лучший путь от {} до {} {}".format(leaving, arriving, day))
        else:
            print("BOT:", translator.translate("I am Finding the best way to go from {} to {} {}".format(leaving, arriving, day), dest=lang).text)
        
    else:
        print("BOT: Finding the best way to go from {} to {} {}".format(leaving, arriving, day))

In [19]:
# Function to wikify the entity
def selenium (url):
    chrome_options = Options() 
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(r'C:/Users/Jason/Downloads/chromedriver_win32/chromedriver.exe', options=chrome_options)
    driver.get(url)
    
    driver.find_element_by_link_text('English').click()
    name= driver.title
    LANGS = [
    'ÁáČčĎďÉéĚěÍíŇňÓóŘřŠšŤťÚúŮůÝýŽž',   # Czech
    'ÄäÖöÜüẞß',                         # German
    'ĄąĆćĘęŁłŃńÓóŚśŹźŻż',               # Polish
    ]
    pattern = r'[A-Z][a-z{langs}]+'.format(langs=''.join(LANGS))
    pattern = re.compile(pattern)
    match = pattern.findall(name)
    match = match[0:-1]
    try:
        if match [1] != None:
            return match[0] + " " + match[1]
    except:
        return match[0]

In [20]:
# Function that captures the highest similarity within the token and the keywords for a language
def highest_similarity_for_keyword(similarity_language, vect):
    last_status = similarity_language[key]
    similarity = cosine_similarity(vect, value)
    if similarity > last_status:
        similarity_language[key] = similarity

In [21]:
# Function that compares the similarities between all languages and returns the highest one overall
def highest_sim_over_language(simbot_en, simbot_sp, simbot_it, simbot_ru, simbot_est, multibot):
    #Eng
    highest_similarity_for_keyword(simbot_en, vect)
    #Sp
    highest_similarity_for_keyword(simbot_sp, vect_sp)
    #It
    highest_similarity_for_keyword(simbot_it, vect_it)
    #Rus
    highest_similarity_for_keyword(simbot_ru, vect_rus)
    #Est
    highest_similarity_for_keyword(simbot_est, vect_est)

    sorted_simil_weather_en = sorted(simbot_en.items(), key=operator.itemgetter(1), reverse=True)
    sorted_simil_weather_sp = sorted(simbot_sp.items(), key=operator.itemgetter(1), reverse=True)
    sorted_simil_weather_it = sorted(simbot_it.items(), key=operator.itemgetter(1), reverse=True)
    sorted_simil_weather_ru = sorted(simbot_ru.items(), key=operator.itemgetter(1), reverse=True)
    sorted_simil_weather_est = sorted(simbot_est.items(), key=operator.itemgetter(1), reverse=True)

    multibot[sorted_simil_weather_en[0][0], 'EN'] = sorted_simil_weather_en[0][1]
    multibot[sorted_simil_weather_sp[0][0], 'SP' ] = sorted_simil_weather_sp[0][1]
    multibot[sorted_simil_weather_it[0][0], 'IT'] = sorted_simil_weather_it[0][1]
    multibot[sorted_simil_weather_ru[0][0], 'RU'] = sorted_simil_weather_ru[0][1]
    multibot[sorted_simil_weather_est[0][0], 'EST'] = sorted_simil_weather_est[0][1]

    # sort the keys by the highest cosine values of all languages
    sorted_simil_weather = sorted(multibot.items(), key=operator.itemgetter(1), reverse=True)
    
    return sorted_simil_weather

In [22]:
# A function that looks for bigram with high cosine similarity for the WikiBot
def wiki_bigram(wh, conf):
    languages = [vect_est, vect_it, vect_rus, vect_sp]
    languages_next = [next_vect_est, next_vect_it, next_vect_rus, next_vect_sp]
    for i in range(len(languages)):
        if cosine_similarity(table_eng[str(wh)].reshape(1,300), languages[i]) + cosine_similarity(table_eng['is'].reshape(1,300), languages_next[i]) > 0.8:
            confidence_wiki.append(conf)
            return True

In [ ]:
# A function that looks for bigram with high cosine similarity for the WikiBot
def wiki_weather(wh, conf):
    languages = [vect_est, vect_it, vect_rus, vect_sp]
    languages_next = [next_vect_est, next_vect_it, next_vect_rus, next_vect_sp]
    for i in range(len(languages)):
        if cosine_similarity(table_eng[str(wh)].reshape(1,300), languages[i]) + cosine_similarity(table_eng['has'].reshape(1,300), languages_next[i]) > 0.8:
            confidence_wiki.append(conf)
            return True

In [39]:
cosine_similarity(table_eng['time'].reshape(1,300), table_it['tempo'].reshape(1,300))

array([[0.6386973]])

In [45]:
cosine_similarity(table_eng['to'].reshape(1,300), table_it['a'].reshape(1,300))

array([[0.48072084]])

In [23]:
# Bigrmas would not work in Russian:
#который сейчас час?
# Just a keyword will not work in Estonian:
# Mis kell on
# -> so let's have a really high cutoff for keywords so they are note confused by when keywords for language like russian
# and bigrams for language like Estonian


# A function that looks for bigram with high cosine similarity for the TimeBot
def time_bigram():
    languages = [vect_est, vect_it, vect_rus, vect_sp]
    languages_next = [next_vect_est, next_vect_it, next_vect_rus, next_vect_sp] 
    for i in range(len(languages)):
        sim1 = cosine_similarity(table_eng['what'].reshape(1,300), languages[i])
        sim2 = cosine_similarity(table_eng['hours'].reshape(1,300), languages_next[i])
        print(sim1 + sim2, 'RESULTS')
        if  sim1 +  sim2 > 0.75:
            confidence_time.append(5)
            return 1
        else:
            return 0

In [1]:
# A function that looks for bigram with high cosine similarity for the WeatherBot
def travel_bigram (FROM, which_city, conf):
    if which_city != None:
        languages = [vect_est, vect_it, vect_rus, vect_sp]
        for i in range(len(languages)):
            if cosine_similarity(table_eng[str(FROM)].reshape(1,300), languages[i]) > conf and next_token.capitalize() == which_city:
                leaving = which_city
                return leaving

In [25]:
# A function that identifies the input language
def language_identifier(sorted_simil):
    if sorted_simil[0][0][1] == 'EN':
        language = 'EN'
    elif sorted_simil[0][0][1] == 'SP':
        language = 'SP'
    elif sorted_simil[0][0][1] == 'IT':
        language = 'IT'
    elif sorted_simil[0][0][1] == 'RU':
        language = 'RU'
    elif sorted_simil[0][0][1] == 'EST':
        language = 'EST'
    return language

In [ ]:
# THE CHATBOT

##### KEYWORDS #############################################################################################################
# keywords for weather_bot
keywords_weather = {'snow':table_eng['snow'], 'rain':table_eng['rain'], 'weather':table_eng['weather'], 
                    'sun':table_eng['sun'], 'hailstorm':table_eng['hailstorm'], 'fog':table_eng['fog'], 
                   'temperature':table_eng['temperature'], 'degree':table_eng['degree'], 'cold': table_eng['cold'],
                   'hot':table_eng['hot'], 'warm':table_eng['warm']}

# keywords for Wikipedia
keywords_wikipedia = {'wikipedia':table_eng['wikipedia'], 'meaning': table_eng['meaning'], 'mean': table_eng['mean'], 
                     'sense': table_eng['sense']}

# keywords for travel_bot
keywords_travel = {'travel': table_eng['travel'], 'trip':table_eng['trip'], 'ride':table_eng['ride'], 'fly':table_eng['fly'],
                  'go':table_eng['go'], 'drive':table_eng['drive'], 'bus':table_eng['bus'], 'plane': table_eng['plane'],
                  'taxi':table_eng['taxi'], 'ferry': table_eng['ferry'], 'car':table_eng['car'], 'train': table_eng['train']}

# keywords for time_bot
keywords_time = {'time': table_eng['time'], 'hours': table_eng['hours'] }

today_key = {'today':table_eng['today'],  'now':table_eng['now']}
tomorrow_key = {'tomorrow':table_eng['tomorrow']}


#############################################################################################################################

# Vectors not found in MUSE
UNK = np.zeros((1, 300))

while True:
    
    #Entities (cities, time, people)
    CITTA = False
    CITTA2 = False
    today = False
    tomorrow = False
    person = False
    yesterday = False
    WHAT = False
    partenza = None
    arrivo = None
    leaving = None
    arriving = None
   
    
    
##### EMPTY DICTIONARIES ####################################################################################################

    # Empty dictionaries for the WEATHER BOT
    confidence_weather = []
    tokens_weather_simil = dict()
    multi_weather = dict()

    similarities_weather_en = {'snow':np.asarray([[0]]), 'rain':np.asarray([[0]]), 'weather':np.asarray([[0]]), 
                    'sun':np.asarray([[0]]), 'hailstorm':np.asarray([[0]]), 'fog':np.asarray([[0]]), 
                   'temperature':np.asarray([[0]]), 'degree':np.asarray([[0]]), 'cold':np.asarray([[0]]),
                   'hot':np.asarray([[0]]), 'warm':np.asarray([[0]])}
    similarities_weather_sp = {'snow':np.asarray([[0]]), 'rain':np.asarray([[0]]), 'weather':np.asarray([[0]]), 
                    'sun':np.asarray([[0]]), 'hailstorm':np.asarray([[0]]), 'fog':np.asarray([[0]]), 
                   'temperature':np.asarray([[0]]), 'degree':np.asarray([[0]]), 'cold':np.asarray([[0]]),
                   'hot':np.asarray([[0]]), 'warm':np.asarray([[0]])}
    similarities_weather_it = {'snow':np.asarray([[0]]), 'rain':np.asarray([[0]]), 'weather':np.asarray([[0]]), 
                    'sun':np.asarray([[0]]), 'hailstorm':np.asarray([[0]]), 'fog':np.asarray([[0]]), 
                   'temperature':np.asarray([[0]]), 'degree':np.asarray([[0]]), 'cold':np.asarray([[0]]),
                   'hot':np.asarray([[0]]), 'warm':np.asarray([[0]])}
    similarities_weather_ru = {'snow':np.asarray([[0]]), 'rain':np.asarray([[0]]), 'weather':np.asarray([[0]]), 
                    'sun':np.asarray([[0]]), 'hailstorm':np.asarray([[0]]), 'fog':np.asarray([[0]]), 
                   'temperature':np.asarray([[0]]), 'degree':np.asarray([[0]]), 'cold':np.asarray([[0]]),
                   'hot':np.asarray([[0]]), 'warm':np.asarray([[0]])}
    similarities_weather_est = {'snow':np.asarray([[0]]), 'rain':np.asarray([[0]]), 'weather':np.asarray([[0]]), 
                    'sun':np.asarray([[0]]), 'hailstorm':np.asarray([[0]]), 'fog':np.asarray([[0]]), 
                   'temperature':np.asarray([[0]]), 'degree':np.asarray([[0]]), 'cold':np.asarray([[0]]),
                   'hot':np.asarray([[0]]), 'warm':np.asarray([[0]])}
    
    # Empty dictionaries for WIKIPEDIA BOT
    confidence_wiki = []
    tokens_wiki_simil = dict()
    multi_wiki = dict()
    similarities_wiki_en = {'wikipedia':np.asarray([[0]]), 'meaning':np.asarray([[0]]), 'mean':np.asarray([[0]]), 
                    'sense':np.asarray([[0]])}#, 'who': np.asarray([[0]]), 'is': np.asarray([[0]])}
    similarities_wiki_sp = {'wikipedia':np.asarray([[0]]), 'meaning':np.asarray([[0]]), 'mean':np.asarray([[0]]), 
                    'sense':np.asarray([[0]])}#, 'who': np.asarray([[0]]), 'is': np.asarray([[0]])}
    similarities_wiki_it = {'wikipedia':np.asarray([[0]]), 'meaning':np.asarray([[0]]), 'mean':np.asarray([[0]]), 
                    'sense':np.asarray([[0]])}#, 'who': np.asarray([[0]]), 'is': np.asarray([[0]])}
    similarities_wiki_ru = {'wikipedia':np.asarray([[0]]), 'meaning':np.asarray([[0]]), 'mean':np.asarray([[0]]), 
                    'sense':np.asarray([[0]])}#, 'who': np.asarray([[0]]), 'is': np.asarray([[0]])}
    similarities_wiki_est = {'wikipedia':np.asarray([[0]]), 'meaning':np.asarray([[0]]), 'mean':np.asarray([[0]]), 
                    'sense':np.asarray([[0]])}#, 'who': np.asarray([[0]]), 'is': np.asarray([[0]])}
    
    # Empty dictionaries for TRAVEL BOT
    confidence_travel = []
    tokens_travel_simil = dict()
    multi_travel = dict()
    similarities_travel_en = {'travel': np.asarray([[0]]), 'trip':np.asarray([[0]]), 'ride':np.asarray([[0]]), 
                'fly':np.asarray([[0]]),'go':np.asarray([[0]]), 'drive':np.asarray([[0]]), 'bus':np.asarray([[0]]),
                'plane': np.asarray([[0]]),'taxi':np.asarray([[0]]), 'ferry': np.asarray([[0]]), 'car':np.asarray([[0]]), 
                'train': np.asarray([[0]])}
    similarities_travel_sp = {'travel': np.asarray([[0]]), 'trip':np.asarray([[0]]), 'ride':np.asarray([[0]]), 
                'fly':np.asarray([[0]]),'go':np.asarray([[0]]), 'drive':np.asarray([[0]]), 'bus':np.asarray([[0]]),
                'plane': np.asarray([[0]]),'taxi':np.asarray([[0]]), 'ferry': np.asarray([[0]]), 'car':np.asarray([[0]]), 
                'train': np.asarray([[0]])}
    similarities_travel_it = {'travel': np.asarray([[0]]), 'trip':np.asarray([[0]]), 'ride':np.asarray([[0]]), 
                'fly':np.asarray([[0]]),'go':np.asarray([[0]]), 'drive':np.asarray([[0]]), 'bus':np.asarray([[0]]),
                'plane': np.asarray([[0]]),'taxi':np.asarray([[0]]), 'ferry': np.asarray([[0]]), 'car':np.asarray([[0]]), 
                'train': np.asarray([[0]])}
    similarities_travel_ru = {'travel': np.asarray([[0]]), 'trip':np.asarray([[0]]), 'ride':np.asarray([[0]]), 
                'fly':np.asarray([[0]]),'go':np.asarray([[0]]), 'drive':np.asarray([[0]]), 'bus':np.asarray([[0]]),
                'plane': np.asarray([[0]]),'taxi':np.asarray([[0]]), 'ferry': np.asarray([[0]]), 'car':np.asarray([[0]]), 
                'train': np.asarray([[0]])}
    similarities_travel_est = {'travel': np.asarray([[0]]), 'trip':np.asarray([[0]]), 'ride':np.asarray([[0]]), 
                'fly':np.asarray([[0]]),'go':np.asarray([[0]]), 'drive':np.asarray([[0]]), 'bus':np.asarray([[0]]),
                'plane': np.asarray([[0]]),'taxi':np.asarray([[0]]), 'ferry': np.asarray([[0]]), 'car':np.asarray([[0]]), 
                'train': np.asarray([[0]])}
    
    # Empty dictionaries for TIME BOT
    confidence_time = []
    multi_time = dict()
    similarities_time_en = {'hours':np.asarray([[0]]), 'time':np.asarray([[0]])}
    similarities_time_sp = {'hours':np.asarray([[0]]), 'time':np.asarray([[0]])}
    similarities_time_it = {'hours':np.asarray([[0]]), 'time':np.asarray([[0]])}
    similarities_time_ru = {'hours':np.asarray([[0]]), 'time':np.asarray([[0]])}
    similarities_time_est = {'hours':np.asarray([[0]]), 'time':np.asarray([[0]])}
     
######## START THE CONVERSATION #############################################################################################
    
    human = input('YOU: ')
    
    # break
    if human == 'bye':
        break
    
######### NER ##############################################################################################################
    LOC = []
    PER = []
    try:
        text = Text(human)
        for sent in text.sentences:
            for e in sent.entities:
                if e.tag == 'I-LOC':
                    LOC.append(e)
                elif e.tag == 'I-PER':
                    PER.append(e)
    except:
        pass
    # Capture if a city is given in the input
    CITY = None
    try: 
        if LOC[0][0] != None:
            CITY = LOC[0][0]
        if LOC[0][1] != None:
            CITY = LOC[0][0] + " "+ LOC[0][1]
    except:
        pass
    if CITY != None:
        CITY = CITY.capitalize()
        CITTA = True
    
    CITY2 = None
    try: 
        if LOC[1][0] != None:
            CITY2 = LOC[1][0]
        if LOC[1][1] != None:
            CITY2 = LOC[1][0] + " "+ LOC[1][1]
    except:
        pass
    if CITY2 != None:
        CITY2 = CITY2.capitalize()
        CITTA2 = True
        
    #Capture if a person is given in the input
    PERSON = None
    try: 
        if PER[0][0] != None:
            PERSON = PER[0][0]
        if PER[0][1] != None:
            PERSON = PER[0][0] + " "+ PER[0][1]
    except:
        pass
    if PERSON != None:
        person = True
        
 
 #############################################################################################################################
    # Language Detector
    try:
        detector = Detector(human).language.name
    except:
        pass
    
    # Tokenize 
    human = human.split()
    hum = []
    
    for tok in human:
        hum.append(re.sub(r'[^\w]', '', tok))
    hum.append('EOS')
    

    for i in range (len(hum)-1):
        # normalize the input
        token = hum[i].lower()
        next_token = hum[i+1].lower()
       
        # Assign the multilingual vectors to the input words
        # assign a zero confidence to stopwords
        vect, vect_sp, vect_it, vect_rus, vect_est = token2multi_vectors(token)
        next_vect, next_vect_sp, next_vect_it, next_vect_rus, next_vect_est = token2multi_vectors(next_token)
        languages = [vect, vect_est, vect_it, vect_rus, vect_sp]
        
         # capture if the time is provided in the input (today, tomorrow)
        oggi = when_def(today_key, 0.5, languages)
        domani = when_def(tomorrow_key, 0.5, languages)
        ieri = when_def(yesterday_key, 0.5, languages)
        if oggi > 0 and oggi > domani:
            today = True
        if domani > 0 and domani > oggi:
            tomorrow = True
        
######## WEATHER BOT ##########################################################################################################

        # "tiempo" and "tempo" have vectors similar to "time" rather than weather. We manually correct this.
        #similarities_weather_sp['tiempo'] = np.array([[0.0]])
        #similarities_weather_it['tempo'] = np.array([[0.0]])
        
        # UNCOMMENT THIS PART TO HAVE THE WEATHERBOT WORKING WT THE KEYWORD "WEATHER" IN THE OTHER LANGUAGES
        
        #if token == 'tiempo':
         #   similarities_weather_sp['tiempo'] = np.array([[0.7]])
        #elif token == 'tempo':
         #   similarities_weather_it['tempo'] = np.array([[0.7]])
       # elif token == 'Погода':
        #    similarities_weather_ru['Погода'] = np.array([[0.7]])
        
        
        # We compute the similarities between the multilingual vectors
        for key, value in keywords_weather.items():
            value = value.reshape(1,300)
            
            # simbot_en, simbot_sp, simbot_it, simbot_ru, simbot_est, multibot):
            sorted_simil_weather = highest_sim_over_language(similarities_weather_en, similarities_weather_sp,
                                    similarities_weather_it, similarities_weather_ru, similarities_weather_est, multi_weather)
            
        # Language identifier
        language_weather = language_identifier(sorted_simil_weather)
        
        # get the highest key-value pair
        if sorted_simil_weather[0][1] > 0.4:
            tokens_weather_simil[sorted_simil_weather[0][0]] = sorted_simil_weather[0][1] 
           
        # check if the highest key-value pair is over the threshold and append for the confidence
        # CONFIDENCE IS GIVEN BY THE SUM OF CAPTURED KEYWORDS ABOVE THE THRESHOLD
        
        if sorted_simil_weather[0][1][0] > 0.46:
            confidence_weather.append(sorted_simil_weather[0][1] +2.6)
        
         
 
 ##### TRAVEL BOT ############################################################################################################

        for key, value in keywords_travel.items():
            value = value.reshape(1,300)
            
            # simbot_en, simbot_sp, simbot_it, simbot_ru, simbot_est, multibot):
            sorted_simil_travel = highest_sim_over_language(similarities_travel_en, similarities_travel_sp,
                                    similarities_travel_it, similarities_travel_ru, similarities_travel_est, multi_travel)
            
        # Language identifier
        language_travel = language_identifier(sorted_simil_travel)
        
        # get the highest key-value pair
        if sorted_simil_travel[0][1] > 0.4:
            tokens_travel_simil[sorted_simil_travel[0][0]] = sorted_simil_travel[0][1] 
           
        # check if the highest key-value pair is over the threshold and append for the confidence
        # CONFIDENCE IS GIVEN BY THE SUM OF CAPTURED KEYWORDS ABOVE THE THRESHOLD
        
        if sorted_simil_travel[0][1][0] > 0.46:
            confidence_travel.append(sorted_simil_travel[0][1] +2.6)
        
        # if we find that FROM and 1st CITY encountered is above the threshold, we set it as the starting point
        # N.B. Unfortunately we cannot make it more general (i.e. from + ANY CITY) bcs the cos_sim of prepositions
        # TO in other languages are higher than the FROM ones (in his case then, they will always be detected as arriving point)
        # but in others as russian FROM is higher than TO
        if CITY2 != None:
            partenza = travel_bigram('from', CITY, 0.35)
            if partenza != None:
                leaving = partenza
            if cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY:
                leaving = CITY
                
            
            
            arrivo = travel_bigram('to', CITY2, 0.45)
            if arrivo != None:
                arriving = arrivo
            if cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY2:
                arriving = CITY2
            
            
            
            if arriving == None:
                arrivo2= travel_bigram('to', CITY, 0.45) 
                if arrivo2 !=None:
                    arriving = arrivo2
                    leaving = CITY2
                if cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY:
                    arriving = CITY
                    leaving = CITY2
            
            
            if leaving == None: 
                partenza2 = travel_bigram('from', CITY2, 0.35)
                if partenza2 != None:
                    leaving = partenza2
                if cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY:
                    leaving = CITY2
                    arriving = CITY
     
        if CITY != None and CITY2 == None:
            partenza = travel_bigram('from', CITY, 0.35)
            if partenza != None:
                leaving = partenza
            if cosine_similarity(table_eng['from'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY:
                leaving = CITY
            
            arrivo = travel_bigram('to', CITY, 0.45)
            if arrivo != None:
                arriving = arrivo
            if cosine_similarity(table_eng['to'].reshape(1,300), vect) > 0.8 and next_token.capitalize() == CITY:
                arriving = CITY
            
        if cosine_similarity(table_eng['going'].reshape(1, 300), vect) == 1:
            confidence_travel.append(-13)
            
        
########## WIKI BOT #########################################################################################################
        
        for key, value in keywords_wikipedia.items():
            value = value.reshape(1,300)
            
            # simbot_en, simbot_sp, simbot_it, simbot_ru, simbot_est, multibot):
            sorted_simil_wiki = highest_sim_over_language(similarities_wiki_en, similarities_wiki_sp,
                                    similarities_wiki_it, similarities_wiki_ru, similarities_wiki_est, multi_wiki)
        # Language identifier
        language = language_identifier(sorted_simil_wiki)
            
        # get the highest key-value pair
        if sorted_simil_wiki[0][1] > 0.4:
            tokens_wiki_simil[sorted_simil_wiki[0][0]] = sorted_simil_wiki[0][1]
        
        # check if the highest key-value pair is over the threshold and append for the confidence
        # CONFIDENCE IS GIVEN BY THE SUM OF CAPTURED KEYWORDS ABOVE THE THRESHOLD
        if sorted_simil_wiki[0][1][0] > 0.4:
            confidence_wiki.append(sorted_simil_wiki[0][1])
   
        if cosine_similarity(table_eng['who'].reshape(1,300), vect) + cosine_similarity(table_eng['is'].reshape(1,300), next_vect) > 1.9:
                confidence_wiki.append(1)
        wiki_bigram('who', 1)
        
        if cosine_similarity(table_eng['what'].reshape(1,300), vect) + cosine_similarity(table_eng['is'].reshape(1,300), next_vect) > 1.9:
                confidence_wiki.append(2.6)
                topic = hum[i+2:-1]
                topic= " ".join(topic)
                WHAT = True
        what = wiki_bigram('what', 1)
        if what == True:
            topic = hum[i+2:-1]
            topic= " ".join(topic)
            WHAT = True
            
        if cosine_similarity(table_eng['where'].reshape(1,300), vect) + cosine_similarity(table_eng['is'].reshape(1,300), next_vect) > 1.9:
                confidence_wiki.append(1)    
        wiki_bigram('where', 1)
    
######## TIME BOT ############################################################################################################
        for key, value in keywords_time.items():
            value = value.reshape(1,300)
            
            # simbot_en, simbot_sp, simbot_it, simbot_ru, simbot_est, multibot):
            sorted_simil_time = highest_sim_over_language(similarities_time_en, similarities_time_sp,
                                    similarities_time_it, similarities_time_ru, similarities_time_est, multi_time)
            
        # Language identifier
        language_time = language_identifier(sorted_simil_time)
           
        # check if the highest key-value pair is over the threshold and append for the confidence
        # CONFIDENCE IS GIVEN BY THE SUM OF CAPTURED KEYWORDS ABOVE THE THRESHOLD
        if sorted_simil_time[0][1][0] > 0.65:
            confidence_time.append(sorted_simil_time[0][1] +2.6)
        
        
        if cosine_similarity(table_eng['what'].reshape(1,300), vect) + cosine_similarity(table_eng['time'].reshape(1,300), next_vect) > 1.9:
                confidence_time.append(9)    
        
        languages = [vect_est, vect_it, vect_rus, vect_sp]
        languages_next = [next_vect_est, next_vect_it, next_vect_rus, next_vect_sp] 
        for i in range(len(languages)):
            sim1 = cosine_similarity(table_eng['what'].reshape(1,300), languages[i])
            sim2 = cosine_similarity(table_eng['hours'].reshape(1,300), languages_next[i])
            if  sim1 +  sim2 > 0.75:
                confidence_time.append(5)
        
    
######## CONFIDENCE OVERALL ###############################################################################################################

    # Compute the confidences and we select the bot with the highest confidence
    conf_weather = sum(confidence_weather)
    conf_wiki = sum(confidence_wiki)
    conf_travel=sum(confidence_travel)
    conf_time = sum(confidence_time)
    if person == True:
        conf_wiki = conf_wiki + 3
    if today or tomorrow or yesterday == True:
        conf_wiki = conf_wiki - 4
    if CITY2 == True:
        conf_travel += 3
    confidence_overall = sorted([conf_weather, conf_travel, conf_wiki, conf_time], reverse=True)
    
######## WEATHER RESPONSE #################################################################################################### 
    if confidence_overall[0] == conf_weather and confidence_overall[0] > 0:
        
        # we use Polyglot only if there is a conflict between IT and SP and we give precedence to Polyglot
        if detector == 'Spanish' and language == 'IT':
            language_weather = 'SP'
        elif detector == 'Italian' and language == 'SP':
            language_weather = 'IT'
        
        # this is to get a different answer according to the keyword, in this case all keywords will give a same answer
        #response = sorted(tokens_weather_simil.items(), key=operator.itemgetter(1), reverse=True)
        
        # We hard-coded dummy answers for all the languages, they provide different answers whether locality and time are given
        # in the input: if they are not given, the function will ask for it.
        if language_weather == 'EN':
            
            #phrase_locality, phrase_when, asnwer_1, answer_2, TODAY, TOMORROW, sorry_answer
            temp= answer_weather(today, tomorrow, "BOT: In which location?", "When?", "today", "tomorrow",CITY,
                           "I am sorry, we don't have information about this location", 'en')
        
        elif language_weather == 'SP':
            try:
                CITY = "_".join(CITY.split())
                website = 'https://es.wikipedia.org/wiki/'+CITY
                CITY = selenium (website)
                answer_weather(today,tomorrow, "En cual ciudad?", "Cuando?",  "hoy", "mañana", CITY,
                                      "Lo siento, no tenemos información sobre esta ciudad", 'es')
            except:
                if CITY != None:
                    CITY = translator.translate(CITY, dest='en').text
                    answer_weather(today,tomorrow,  "En cual ciudad?", "Cuando?",  "hoy", "mañana", CITY,
                                      "Lo siento, no tenemos información sobre esta ciudad", 'es')
                    
                else:
                    answer_weather(today,tomorrow,  "En cual ciudad?", "Cuando?",  "hoy", "mañana", CITY,
                                      "Lo siento, no tenemos información sobre esta ciudad", 'es')
                
        elif language_weather == 'IT':
            try:
                CITY = "_".join(CITY.split())
                website = 'https://it.wikipedia.org/wiki/'+CITY
                CITY = selenium (website)
                answer_weather(today,tomorrow,"In quale citta'?", 'Quando?', "oggi", "domani", CITY,
                           "Mi dispiace, non disponiamo di informazioni riguardo a questa citta'", 'it')   
            except:
                if CITY != None:
                    CITY = translator.translate(CITY, dest='en').text
                    answer_weather(today,tomorrow, "In quale citta'?", 'Quando?', "oggi", "domani", CITY,
                           "Mi dispiace, non disponiamo di informazioni riguardo a questa citta'", "it")
                else:
                    answer_weather(today,tomorrow, "In quale citta'?", 'Quando?', "oggi", "domani", CITY,
                           "Mi dispiace, non disponiamo di informazioni riguardo a questa citta'", 'it')
                
        elif language_weather == 'RU':
            try:
                CITY = "_".join(CITY.split())
                website = 'https://ru.wikipedia.org/wiki/'+CITY
                CITY = selenium (website)
                answer_weather(today,tomorrow,  "в каком городе?", 'когда?', "сегодня", "завтра", CITY,
                               "Извините, у нас нет информации об этом городе", 'ru')   
            except:
                if CITY != None:
                    CITY = translator.translate(CITY, dest='en').text 
                    answer_weather(today,tomorrow, "в каком городе?", 'когда?', "сегодня", "завтра", CITY,
                                   "Извините, у нас нет информации об этом городе", 'ru')    
                else:
                    answer_weather(today,tomorrow,"в каком городе?", 'когда?',  "сегодня", "завтра", CITY,
                                   "Извините, у нас нет информации об этом городе", 'ru')  
                    
        elif language_weather == 'EST':
            try:
                CITY = "_".join(CITY.split())
                website = 'https://et.wikipedia.org/wiki/'+CITY
                CITY = selenium (website)
                answer_weather(today,tomorrow,  "Millises linnas?", 'Kui', "täna", "homme", CITY,
                           "Vabandust, meil ei ole selle linna kohta teavet", 'et')   
            except:
                if CITY != None:
                    CITY = translator.translate(CITY, dest='en').text
                    answer_weather(today,tomorrow, "Millises linnas?", 'Kui?', "täna", "homme", CITY,
                           "Vabandust, meil ei ole selle linna kohta teavet", 'et')  
                else:
                    answer_weather(today,tomorrow,  "Millises linnas?", 'Kui?',"täna", "homme", CITY,
                           "Vabandust, meil ei ole selle linna kohta teavet", 'et')  
                    
####### TRAVEL ANSWER ######################################################################################################
    elif confidence_overall[0] == conf_travel and confidence_overall[0] > 0:
        # we use Polyglot only if there is a conflict between IT and SP and we give precedence to Polyglot
        if detector == 'Spanish' and language_travel == 'IT':
            language_travel = 'SP'
        elif detector == 'Italian' and language_travel == 'SP':
            language_travel = 'IT'
        
        if language_travel == 'EN':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'en', "BOT: Which is your destination?", 
            "BOT: From which city would you like to leave?", "BOT: I am sorry, the city you inserted doesn't coincide with the one you told me earlier",
            "BOT: When?", "today", "tomorrow")
        elif language_travel == 'SP':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'es', "BOT: ¿Cuál es tu destino?",
            "BOT: ¿De qué ciudad te gustaría irte?", "BOT: Lo siento, la ciudad insertada no coincide con la que me dijiste antes",
             "BOT: Cuando?", "hoy", "mañana")
        elif language_travel == 'IT':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'it', "BOT: qual è la tua destinazione?", 
             "BOT: Da quale città ti piacerebbe partire?", "BOT: Mi dispiace, la città che hai digitato non corrisponde a quella inserita in precedenza",
            "BOT: Quando?", "oggi", "domani")
        elif language_travel == 'EST':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'et', "BOT: mis on teie sihtkoht?",
            "BOT: milline linn sooviksite lahkuda?", "BOT: Mul on kahju, et sisestatud linn ei lange kokku sellega, mida sa varem ütlesid",
             "BOT: Millal?", "täna", "homme")
        elif language_travel == 'RU':
            answer_travel(CITY, CITY2, leaving, arriving, today, tomorrow, 'ru', "BOT: Куда ты направляешься",
            "BОТ: из какого города вы хотели бы уехать?", "БОТ: извините, введенный вами город не совпадает с тем, который вы мне сказали ранее",
            "BОТ: Когда?", "Сегодня", "Завтра")      
            
        

######## WIKIPEDIA ANSWER ######################################################################################################
    elif confidence_overall[0] == conf_wiki and confidence_overall[0] > 0:
        
        if detector == 'Spanish' and language == 'IT':
            language = 'SP'
        elif detector == 'Italian' and language == 'SP':
            language = 'IT'
            
        # this is to get a different answer according to the keyword, in this case all keywords will give a same answer
        #response = sorted(tokens_wiki_simil.items(), key=operator.itemgetter(1), reverse=True)
        
        # We hard-coded dummy answers for all the languages, they provide different answers whether locality and time are given
        # in the input: if they are not given, the function will ask for it.
      
        
        if language == 'EN':
            wikipedia.set_lang("en")
            answer_wiki("BOT: Sorry I am not sure I understood, which topic were you looking for?",
                        "BOT: Sorry we don't have information about")

        elif language == 'SP':
            wikipedia.set_lang("es")
            answer_wiki("BOT: Lo siento, no estoy seguro de haber entendido, ¿qué tema buscabas?",
                       "Lo sentimos, no tenemos información sobre")
            
        elif language == 'RU':
            wikipedia.set_lang("ru")
            answer_wiki("Извините, я не уверен, что понял, какую тему вы искали?",
                       "Извините, у нас нет информации о")
        
        elif language == 'EST':
            wikipedia.set_lang("et")
            answer_wiki("Vabandust, et ma ei ole kindel, et sain aru, millist teemat otsisite?",
                       "Vabandame, et meil pole teavet")
         
        elif language == 'IT':
            wikipedia.set_lang("it")
            answer_wiki("Mi dispiace non sono sicuro di aver capito, quale argomento stavi cercando?",
                       "Ci dispiace, non abbiamo informazioni su")
        
##### TIME ANSWER ############################################################################################################
    elif confidence_overall[0] == conf_time and confidence_overall[0] > 0:
        if detector == 'Spanish' and language == 'IT':
            language = 'SP'
        elif detector == 'Italian' and language == 'SP':
            language = 'IT'
        
        if language == 'EN':
            print("BOT: It's", datetime.now().strftime("%H:%M"))
            
        elif language == 'SP':
            print ("BOT: Son las", datetime.now().strftime("%H:%M"))
            
        elif language == 'RU':
            print("BOT: Это", datetime.now().strftime("%H:%M"), "вечера")
           
        elif language == 'EST':
            print("BOT: See on", datetime.now().strftime("%H:%M"))
           
        elif language == 'IT':
            print("BOT: Sono le", datetime.now().strftime("%H:%M"))
        
        
    # in case the threshold is not reached
    else:
        print("Sorry I didn't understand, could you repeat in other words?")
        